- Chargement du fichier qui contient la description de la pose "keypoints"
- Créer un tenseur topology pour définir comment les keypoints sont liés entre eux.

In [1]:
import json
import trt_pose.coco
import math

with open('human_pose.json', 'r') as f:
    human_pose = json.load(f)

topology = trt_pose.coco.coco_category_to_topology(human_pose)

- Définir la résolution de l'entrée 224x224
- Initialisé l'entrée avec un example qui contient que des zeros

In [2]:
import torch
WIDTH = 224
HEIGHT = 224
data = torch.zeros((1, 3, HEIGHT, WIDTH)).cuda()

- charger les poids du réseau optimisé
Remarque: ce réseau a été optimisé en utilisant torch2trt

In [3]:
from torch2trt import TRTModule
OPTIMIZED_MODEL = 'mon_model.pth'
model_trt = TRTModule()
model_trt.load_state_dict(torch.load(OPTIMIZED_MODEL))

<All keys matched successfully>

- Mesurer le performance du réseau, le réseultat obtenu est en FPS 

In [4]:
import time

t0 = time.time()
torch.cuda.current_stream().synchronize()
for i in range(50):
    y = model_trt(data)
torch.cuda.current_stream().synchronize()
t1 = time.time()

print(50.0 / (t1 - t0))

7.53176273642975


 - Importer les bibliothéques pour le traitement d'images (cv2), ainsi pour le prétraitement (torchvision.transforms,PIL.image)
 - Définir les tenseurs de la moyenne, écart-type pour la normalisation de l'image et créer une allocation pour ces tenseur dans le GPU (torch.device('cuda'))
 la fonction preprocess:
     - Convertit l'image en RGB
     - Convertir l'image en PIL image dans le but de le transformer après en tenseur (voir ligne d'après)
     - Normaliser l'image

In [5]:
import cv2
import torchvision.transforms as transforms
import PIL.Image

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda()
device = torch.device('cuda')

def preprocess(image):
    global device
    device = torch.device('cuda')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

- "parse_objects" classe pour renvoyer les keypoints trouvés à partir du réseau de neurones
- "draw_objects" classe pour dessiner ces points

In [6]:
from trt_pose.draw_objects import DrawObjects
from trt_pose.parse_objects import ParseObjects

parse_objects = ParseObjects(topology)
draw_objects = DrawObjects(topology)

- utiliser la biblithéque jetcam (sur nvidia seulement) pour récupérer une image BGR 

In [7]:
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg

camera = CSICamera(width=WIDTH, height=HEIGHT,capture_fps=2)

camera.running = True

RuntimeError: Could not initialize camera.  Please see error trace.

In [8]:
import cv2
cap = cv2.VideoCapture(0)

- Initialiser 4 tenseurs pytorch pour enregistrer les coordonnées des keypoints récupérés

In [9]:
# intialisation des tenseurs pytorch pour le stockage 
x,y,x1,y1 = torch.zeros([1,18]),torch.zeros([1,18]),torch.zeros([1,18]),torch.zeros([1,18])

#initialisation du tenseur pour le comptage
cpt = torch.zeros([1,4])


def stocker(tenseur_x,tenseur_y):
    global x1,y1
    x1 = torch.cat((x1,tenseur_x),0)
    y1 = torch.cat((y1,tenseur_y),0)
    return x1,y1



In [10]:
from Mouvements import Mouvements as mvt
from Manipulation import Manipulation as manip
#création des objets instances

mouvement = mvt()
manipulations = manip()


Fonction pour récupérer les keypoints 

In [11]:
def get_keypoints(image, human_pose, topology, object_counts, objects, normalized_peaks):
    global x,y

    height = image.shape[0]
    width = image.shape[1]
    count = int(object_counts[0])# il y a combien de personnes détéctés
    
    
    for i in range(count):# parcourir les personnes détéctées
        obj = objects[0][i] # obj determine les points clés détéctés
        # obj est -1 si le keypoint n'existe pas et 0 s'il existe
        C = obj.shape[0] # le nombre des keypoints (toujours C= 18)
        for j in range(C): #parcourir les 18 points clés j est le nombre de l'indice du keypoint
                k = int(obj[j]) 
                if k >= 0: #si le keypoint existe
                    peak = normalized_peaks[0][j][k] #on associe le keypoint qui existe à l'indice du keypoint J
                    x[0][j] = round(float(peak[1]) * width)
                    y[0][j] = round(float(peak[0]) * height)
                else:
                    #quand on a pas une detection on rempli par la derniere valeur qu'on a obtenu
                    x[0][j] = x1[-1][j]
                    y[0][j] = y1[-1][j]
                    
    # Stocker les Coordonnées                
    stocker(x,y) 
    # Renvoyer l'angle du bras gauche
    angle_gauche = manipulations.getAngle(x1,y1,9,7,5)
    # Détécter l'uppercut gauche
    uppercut_gauche = mouvement.uppercut(y1,angle_gauche,'gauche',cpt)
    # Renvoyer l'angle du bras droit
    angle_droite = manipulations.getAngle(x1,y1,10,8,6)
    # Détécter l'uppercut droite
    uppercut_droite = mouvement.uppercut(y1,angle_droite,'droite',cpt)
    #Detecter le JAB
    JAB = mouvement.JAB(angle_gauche,y1,'gauche',cpt)
    #Detecter le contre
    Cross = mouvement.JAB(angle_droite,y1,'droite',cpt)
    #rajouter les coups dans le tenseur combinaison (comb)
    manipulations.combinaisons(JAB,Cross,uppercut_gauche,uppercut_droite)
    
    


Fonction d'éxécution pour charger le nouvelle l'image,effectuer un pretraitement,detecter les keypoints, convertir l'image en JPEG et enfin appeler la fonction get_keypoints

In [12]:
def execute(change):
    image = change['new']
    data = preprocess(image)
    cmap, paf = model_trt(data)
    cmap, paf = cmap.detach().cpu(), paf.detach().cpu()
    counts, objects, peaks = parse_objects(cmap, paf)
    draw_objects(image, counts, objects, peaks)
    image_w.value = bgr8_to_jpeg(image[:, ::-1, :])
    keypoints = get_keypoints(image, human_pose, topology, counts, objects,peaks)

- une interface pour la visualisation en temps réel

In [13]:
import ipywidgets
from IPython.display import display
image_w = ipywidgets.Image(format='jpeg',width = 500,height = 300)
display(image_w)

Image(value=b'', format='jpeg', height='300', width='500')

Lancer la caméra

In [ ]:
#camera.observe(execute, names='value')

error: OpenCV(4.4.0) /tmp/pip-install-um0km5dj/opencv-python_84492e22e5ef44aa8614e1eba3873e6c/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


Arréter la caméra

In [ ]:
#camera.unobserve_all()


### 

### 

In [ ]:
#bilan des mouvements
#manipulations.statistiques(cpt,manipulations.comb)


### 